In [1]:
#https://stackoverflow.com/questions/52771328/plotly-chart-not-showing-in-jupyter-notebook

In [2]:
#Immportation des données de metagenomique
import pandas as pd

df = pd.read_csv("1335.vdb.tab", sep ='\t')

#decoupage des noms des bacteries

l1= []
l2= []
l3= []
l4= []
l5= []
l6= []

for nom in df['OTUConTaxonomy'].tolist():
    l1.append( nom.split(";")[0].split("(")[0])
    l2.append( nom.split(";")[1].split("(")[0])
    l3.append( nom.split(";")[2].split("(")[0])
    l4.append( nom.split(";")[3].split("(")[0])
    l5.append( nom.split(";")[4].split("(")[0])
    l6.append( nom.split(";")[5].split("(")[0])

df['regne'] = l1
df['embranchement']= l2
df['classe'] = l3
df['ordre']= l4
df['famille']= l5
df['genre']= l6

df.head()

,OTUNumber,OTUConTaxonomy,door_in_1,door_in_2,faucet_handle_1,faucet_handle_2,sink_floor_1,sink_floor_2,soap_dispenser_1,stall_in_1,...,toilet_flush_handle_1,toilet_flush_handle_2,toilet_seat_1,toilet_seat_2,regne,embranchement,classe,ordre,famille,genre
0,Otu00001,Bacteria(100);Actinobacteria(100);Actinobacter...,1381,1032,335,577,38,36,1115,598,...,13,329,76,156,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Propionibacteriaceae,Propionibacterium
1,Otu00002,Bacteria(100);Proteobacteria(100);Gammaproteob...,20,17,1686,3,411,564,128,9,...,0,25,258,0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Enhydrobacter
2,Otu00003,Bacteria(100);Actinobacteria(100);Actinobacter...,8,47,6,418,15,8,84,45,...,3,283,41,1059,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Corynebacteriaceae,Corynebacterium
3,Otu00004,Bacteria(100);Actinobacteria(100);Actinobacter...,6,27,5,6,452,275,63,0,...,9,51,5,3,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Micrococcaceae,Arthrobacter
4,Otu00005,Bacteria(100);Actinobacteria(100);Actinobacter...,0,29,0,648,2,8,3,32,...,0,206,7,95,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Corynebacteriaceae,Corynebacterium


In [24]:
import plotly.graph_objects as go

df_short = df[['door_in_1','door_in_2','faucet_handle_1','faucet_handle_2','sink_floor_1','sink_floor_2','soap_dispenser_1','stall_in_1','toilet_floor_1','toilet_floor_2','toilet_flush_handle_1','toilet_flush_handle_2','toilet_seat_1','toilet_seat_2']]

#remove line with 0

df_short_clean = df_short.loc[(df.sum(axis=1) != 0)]

fig = go.Figure(data=go.Heatmap(
                   z=df_short_clean.to_numpy(),
                   x=df_short_clean.columns.values,
                   y=df['OTUConTaxonomy'].to_list(),
                   hoverongaps = False))
fig.show()

In [22]:
#Objectif : Visualisation de la repartition des differents embranchements au sein des differents echantillons a l'aide d'un barplot

matrice=df.groupby(by="embranchement").sum()


indexNamesArr = matrice.index.values
indexNamesArr
listOfRowIndexLabels = list(indexNamesArr)
matrice.to_numpy
columnsNamesArr = matrice.columns.values
listOfColumnNames = list(columnsNamesArr)



fig = px.bar(matrice.transpose(), x=columnsNamesArr, y=listOfRowIndexLabels, color_discrete_sequence=px.colors.sequential.Viridis, title="Répartition des differents embranchements")
fig.show()
